In [1]:
import numpy as np
import pandas as pd
import math
import re

In [2]:
FILE_NAME = "test.txt"
#FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False

In [3]:
CHAR_MAP = {".":0, "/":1,"\\":2,"|":3,"-":4}

def getCharFromNum(num):
    for x,y in CHAR_MAP.items():
        if y == num:
            return x
            

In [4]:
def buildArrayFromLines(lines):
    
    tmp_arr = []
    for line in lines:
        tmp_line = []    
        
        for c in line:
            if c != "\n":
                tmp_line += [CHAR_MAP[c]]
        tmp_arr.append(tmp_line)
    
    
    return np.array(tmp_arr)
    

In [5]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    my_arr = buildArrayFromLines(lines)
    
    
    return my_arr


In [6]:
def decideNextStep(cell,direct):
    if DIAG: print(cell)
    res = []    
    if cell == 0:
        res  = [direct]
    elif cell == 1:
        res = [(-direct[1],-direct[0])]
    elif cell == 2:
        res = [(direct[1],direct[0])]
    elif cell == 3:
        if direct[1] == 0:
            res  = [direct]
        else:
            res = [(-1,0),(1,0)]
    elif cell == 4:
        if direct[0] == 0:
            res  = [direct]
        else:
            res = [(0,-1),(0,1)]
    else:
        print("What the hell?")
    return res

assert decideNextStep(1,(0,1)) == [(-1,0)]
assert decideNextStep(1,(-1,0)) == [(0,1)]
assert decideNextStep(1,(1,0)) == [(0,-1)]
assert decideNextStep(1,(0,-1)) == [(1,0)]

assert decideNextStep(2,(0,1)) == [(1,0)]
assert decideNextStep(2,(-1,0)) == [(0,-1)]
assert decideNextStep(2,(1,0)) == [(0,1)]
assert decideNextStep(2,(0,-1)) == [(-1,0)]

assert decideNextStep(3,(0,1)) == [(-1,0),(1,0)]
assert decideNextStep(3,(-1,0)) == [(-1,0)]
assert decideNextStep(3,(1,0)) == [(1,0)]
assert decideNextStep(3,(0,-1)) ==  [(-1,0),(1,0)]

assert decideNextStep(4,(0,1)) == [(0,1)] #[(-1,0),(1,0)]
assert decideNextStep(4,(-1,0)) == [(0,-1),(0,1)]
assert decideNextStep(4,(1,0)) == [(0,-1),(0,1)]
assert decideNextStep(4,(0,-1)) ==  [(0,-1)] #[(0,-1),(0,1)]

assert decideNextStep(0,(0,1)) == [(0,1)]
assert decideNextStep(0,(-1,0)) == [(-1,0)]
assert decideNextStep(0,(1,0)) == [(1,0)]
assert decideNextStep(0,(0,-1)) == [(0,-1)]

In [7]:
#STEPS_PROCESSED = set()
#STEPS_TO_PROCESS = set()

In [8]:
def processStep(inMap,outMap,x,y,direct,STEPS_PROCESSED,STEPS_TO_PROCESS):
    """
    direct is (,) and a direction we had coming into the cell
    """
    if DIAG: print(">> step to process:",x,y,direct)
    shape_X,shape_Y = inMap.shape
    if not(x in range(0,shape_X) and y in range(0,shape_X)):
        # we had stepped  off the map
        if DIAG: print("off the map")
        pass
    elif (x,y,direct) in STEPS_PROCESSED:
        #we've already walked through this cell in this direction. Nothing to do
        if DIAG: print("already seen")
        pass
    else:
        # mark cell as hot
        outMap[x,y]=1
        
        STEPS_PROCESSED.add((x,y,direct))
        # STEPS_TO_PROCESS.REMOVE((x,y,direct)) it's already been popped
        if DIAG: print("processing")
        next_steps = decideNextStep(inMap[x,y],direct)
        if DIAG: print("  Have some next steps: ",next_steps)
        for step in next_steps:
            nn = (x+step[0],y+step[1],step)
            if nn in STEPS_PROCESSED:
                pass
            else:
                STEPS_TO_PROCESS.add(nn)
                    
    return outMap

In [9]:
def processMap(inMap,starting_step):
    #print(mirror_map)
    energ_map = np.zeros(inMap.shape)    

    STEPS_PROCESSED  = set([])
    STEPS_TO_PROCESS  = set([starting_step])
    counter = 0 
    while len(STEPS_TO_PROCESS) > 0 and counter < 10000000:
        move = STEPS_TO_PROCESS.pop()
        if DIAG: print("Outer loop: ", move, "counter: ",counter)
        energ_map= processStep(mirror_map,energ_map,*move,STEPS_PROCESSED,STEPS_TO_PROCESS)
    
        counter+=1
    
    res = sum(sum(energ_map))
    if DIAG: print(res)
    return res
    


In [12]:
def getBestActivation(map_to_process,starting_steps):
    res = []
    for step in starting_steps:
        res_sing = processMap(mirror_map,step)
        res+=[res_sing]

    #print(res)
    return max(res)

In [13]:
def getActivationTopLeftCorner(map_to_process):
    starting_steps = [(0,0,(0,1))]
    return getBestActivation(map_to_process,starting_steps)

In [16]:
def getActivationAllEdge(map_to_process):
    shape_X,shape_Y = mirror_map.shape

    starting_steps = [(0,x,(1,0)) for x in range(0,shape_Y)]
    starting_steps += [(shape_X-1,x,(-1,0)) for x in range(0,shape_Y)]
    starting_steps += [(x,0,(0,1)) for x in range(0,shape_X)]
    starting_steps += [(x,shape_Y-1,(0,-1)) for x in range(0,shape_X)]

    return getBestActivation(map_to_process,starting_steps)

In [17]:
# import sys


# np.set_printoptions(threshold=sys.maxsize)

mirror_map = parseFile(FILE_NAME)

resA = getActivationTopLeftCorner(mirror_map)
resB = getActivationAllEdge(mirror_map)

print("Result part A:",resA)
print("Result part B:",resB)

Result part A: 8112.0
Result part B: 8314.0


# Testing things out

In [11]:
(1,1)+(-1,-1)

(1, 1, -1, -1)